In [1]:
import json
import os
from os import listdir
from os.path import isfile, join
import random
import time
import datetime
import csv

In [2]:
# Model libraries
from sentence_transformers import SentenceTransformer
import sentencepiece
from scipy.spatial import distance
import numpy as np

from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

c:\Users\allie\Documents\GitHub\policy-data-analyzer\vvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading the individual policy sentences jsons

In [3]:
policy_dict = {}
objs = []
language = "spanish"
os.chdir("C:/Users/Allie/Documents/GitHub/policy-data-analyzer/tasks/")
#C:\Users\allie\Documents\GitHub\policy-data-analyzer\tasks\text_preprocessing\output\new
mypath = "./text_preprocessing/output/new/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for f in onlyfiles:
    #serializedObject = obj.get()['Body'].read()
    #policy_dict = {**policy_dict, **json.loads(serializedObject)}
    with open(join(mypath, f), 'r') as file:
        data = json.load(file)
    policy_dict = {**policy_dict, **data}


In [4]:
policy_dict.keys()

dict_keys(['2019CVE 1713470_Chile', 'CONAFOROperations_Mexico', 'Constitution_ElSalvador', 'CreditoGanadero_Mexico', 'Decreto 51_Chile', 'Decreto 95_Chile', 'DECRETO008-2005Marco-del-Sistema-Nacional-de-Gestión-Ambiental', 'Decreto101-96_Ley Forestal_Guatemal', 'Decreto102-2001_Peru', 'Decreto2-2015_PROBOSQUE_Guatemala', 'Decreto272_Cafe', 'Decreto864_ApoyoCafe', 'Decreto8_Chile', 'Decreto_109_Guatemala', 'Decreto_51_PINPEP_Guatemala', 'Decreto_MINAM_Peru', 'educacion_ambiental_el_salvador', 'gua60538', 'LEGISLATIVOS-AMERICA-LATINA', 'Ley 20412_Chile', 'Ley 29325  FISCALIZACIÓN AMBIENTAL', 'Ley de areas naturales protegidas_ElSalvador', 'Ley del medio ambiente_ElSalvador', 'Ley Especial Cafe_ElSalvador', 'Ley Forestal_ElSalvador', 'LEY GENERAL DESARROLLO FORESTAL_Mexico', 'Ley Turismo_ElSalvador', 'Ley-General-Ambiente_Peru', 'LEY-SOBRE-CHILE', 'Ley30573 - Reestructuración Agraria Especial_Peru', 'LeyAGROINDUSTRIA_ElSalvador', 'LeyCiencia_Mexico', 'LeyElectricidad_ElSalvador', 'LeyP

List of all the sentences' names

In [5]:
def labeled_sentences_from_dataset(dataset):
    sentence_tags_dict = {}

    for document in dataset.values():
        sentence_tags_dict.update(document['sentences'])

    return sentence_tags_dict

sentences = labeled_sentences_from_dataset(policy_dict)

Just checking that it has all 61 pdfs

In [6]:
#len(sentences.keys())
i = 0
for j in sentences.keys():
    if j[-2:] == '_0':
        i+=1
print(i)

61


In [7]:
def create_sentence_embeddings(model, sentences_dict, file):
    embeddings = {}
    for sentence_id, sentence_map in sentences_dict.items():
        embeddings[sentence_id] = model.encode(sentence_map['text'].lower(), show_progress_bar=False)
        
    return embeddings
    #    with open(file, 'w') as fp:
    #        json.dump(embeddings, fp, cls = NumpyArrayEncoder)

In [8]:
sample_sentence_ids = random.sample(list(sentences), 10)
sample_sentences = {}
for s_id in sample_sentence_ids:
    sample_sentences.update({s_id: sentences[s_id]})

In [9]:
sample_sentences

{'CONAFOROperations_Mexico_sent_2415': {'text': 'Forma parte esencial del objeto del presente instrumento juridico, el que la o el "ASESOR(A) TECNICO(A)" coadyuve a la ejecucion y total conclusion de "EL PROYECTO" , de conformidad con la normatividad aplicable a "EL PROYECTO" .',
  'label': []},
 'CONAFOROperations_Mexico_sent_2387': {'text': 'Con base en sus usos y costumbres, su(s) representante(s) acredita(n) tal caracter con el documento _____________________________ y cuentan con los permisos necesarios del pueblo o la comunidad indigena para que en su nombre y representacion se obliguen y realicen todas las acciones que se deriven del presente instrumento juridico.',
  'label': []},
 'LEY-SOBRE-CHILE_sent_313': {'text': 'Pronunciarse previamente sobre los proyectos de Reglamento y sus modi-ficaciones, emitir opinion sobre la eje -cucion de la ley y proponer las adecua-ciones normativas legales y reglamen-tarias que estime necesarias;c. Formular observaciones a las politicas que e

will want to update the transformer used
originally did the sample of 10 sentences which took about 4 seconds
Now doing all sentences which will take longer
I wonder if im going to get a memory error?

In [27]:
Ti = time.perf_counter()

# We will use only one transformer to compute embeddings
transformer_name = 'xlm-r-bert-base-nli-stsb-mean-tokens'

path = "./data_augmentation/output/new/"
today = datetime.date.today()
today = today.strftime('%Y-%m-%d')
filename = "Embeddings_" + today + "_ES.json"
file = path + filename


model = SentenceTransformer(transformer_name, device="cuda")
embs = create_sentence_embeddings(model, sample_sentences, file)
#embs = create_sentence_embeddings(model, sentences, file)

Tf = time.perf_counter()

print(f"The building of a sentence embedding database in the two(?) current models has taken {Tf - Ti:0.4f} seconds")

25239
The building of a sentence embedding database in the two(?) current models has taken 2.1906 seconds


In [11]:
embs.keys()
#len(embs["Ley_Fauna_Silvestre_Peru_sent_69"])

dict_keys(['CONAFOROperations_Mexico_sent_2415', 'CONAFOROperations_Mexico_sent_2387', 'LEY-SOBRE-CHILE_sent_313', 'Ley_Fauna_Silvestre_Peru_sent_2412', 'CONAFOROperations_Mexico_sent_750', 'LEY GENERAL DESARROLLO FORESTAL_Mexico_sent_526', 'Mexico_CONAFOR_ plan 2025_sent_61', 'LeyCiencia_Mexico_sent_437', 'Resolucion_1.01-2007_Guatemala_sent_132', 'Ley_Fauna_Silvestre_Peru_sent_650'])

In [12]:
with open(file, 'a') as fp:
    json.dump(embs, fp, cls = NumpyArrayEncoder)

In [13]:
def sentence_similarity_search(model, queries, sentence_embeddings, sentences, similarity_limit, results_limit, filename):
    results = {}
    for query in queries:
        Ti = time.perf_counter()
        similarities = get_distance(model, sentence_embeddings, sentences, query, similarity_limit)
        results[query] = similarities[0:results_limit]#results[transformer][query] = similarities[0:results_limit]
        Tf = time.perf_counter()
        print(f"similarity search for query {query} has been done in {Tf - Ti:0.4f} seconds")

    path = "../output/"
    filename = filename + ".json"
    file = path + filename
    #with open(file, 'w+') as fp:
    #    json.dump(results, fp, indent=4)
    return results

# This function helps debugging misspelling in the values of the dictionary
def check_dictionary_values(dictionary):
    check_country = {}
    check_incentive = {}
    for key, value in dictionary.items():
        incentive, country = value.split("-")
        check_incentive[incentive] = 0
        check_country[country] = 0
    print(check_incentive)
    print(check_country)

def get_distance( model, sentence_emb, sentences_dict, query, similarity_treshold):
    query_embedding = model.encode(query.lower(), show_progress_bar=False, device='cuda')
    highlights = []
    for sentence in sentences_dict.keys():
        try:
            sentence_embedding = sentence_emb[sentence]
            score = 1 - distance.cosine(sentence_embedding, query_embedding)
            if score > similarity_treshold:
                highlights.append([sentence, score, sentences_dict[sentence]['text']])
        except KeyError as err:
            print(sentence)
            print(sentence_emb.keys())
            print(err)
    highlights = sorted(highlights, key = lambda x : x[1], reverse = True)
    return highlights

In [14]:
queries_dict = {
    "Otorgamiento de estímulos crediticios por parte de el estado" : "Credit-México",
"Estos créditos podrían beneficiar a sistemas productivos asociados a la pequeña y mediana producción" : "Credit-Perú",
"Se asocia con créditos de enlace del Banco del Estado" : "Credit-Chile", 
"Acceso al programa de garantía crediticia para la actividad económica" : "Credit-Guatemala",
"El banco establecerá líneas de crédito para que el sistema financiero apoye la pequeña, mediana y microempresa" : "Credit-El Salvador",
"Dentro de los incentivos económicos se podrá crear un bono para retribuir a los propietarios por los bienes y servicios generados." : "Direct_payment-México",
"Acceso a los fondos forestales para el pago de actividad" : "Direct_payment-Perú",
"Se bonificará el 90% de los costos de repoblación para las primeras 15 hectáreas y de un 75% respecto las restantes" : "Direct_payment-Chile",
"El estado dará un incentivo que se pagará una sola vez a los propietarios forestales" : "Direct_payment-Guatemala",
"Incentivos en dinero para cubrir los costos directos e indirectos del establecimiento y manejo de areas de producción" : "Direct_payment-El Salvador",
"Toda persona física o moral que cause daños estará obligada a repararlo o compensarlo" : "Fine-México",
"Disminuir los riesgos para el inversionista implementando mecanismos de aseguramiento" : "Guarantee-México",
"Podrá garantizarse el cumplimiento de la actividad mediante fianza otorgada a favor del estado por cualquiera de las afianzadoras legalmente autorizadas." : "Guarantee-Guatemala",
"El sujeto de derecho podrá recibir insumos para la instalación y operación de infraestructuras para la actividad económica." : "Supplies-México",
"Se facilitará el soporte técnico a  través de la utilización de guías, manuales, protocolos, paquetes tecnológicos, procedimientos, entre otros." : "Supplies-Perú",
"Se concederán incentivos en especie para fomentar la actividad en forma de insumos" : "Supplies-El Salvador",
"Se otorgarán incentivos fiscales para la actividad primaria y también la actividad de transformación" : "Tax_deduction-México",
"De acuerdo con los lineamientos aprobados se concederá un 25% de descuento en el pago del derecho de aprovechamiento" : "Tax_deduction-Perú",
"Las bonificaciones percibidas o devengadas se considerarán como ingresos diferidos en el pasivo circulante y no se incluirán para el cálculo de la tasa adicional ni constituirán renta para ningún efecto legal hasta el momento en que se efectúe la explotación o venta" : "Tax_deduction-Chile",
"Los contratistas que suscriban contratos de exploración y/o explotación, quedan exentos de cualquier impuesto sobre los dividendos, participaciones y utilidades" : "Tax_deduction-Guatemala",
"Exención de los derechos e impuestos, incluyendo el Impuesto a la Transferencia de Bienes Muebles y a la Prestación de Servicios, en la importación de sus bienes, equipos y accesorios, maquinaria, vehículos, aeronaves o embarcaciones" : "Tax_deduction-El Salvador",
"Se facilitará formación Permanente Además del acompañamiento técnico, los sujetos de derecho participarán en un proceso permanente de formación a lo largo de todo el año, que les permita enriquecer sus habilidades y capacidades " : "Technical_assistance-México",
"Contribuir en la promoción para la gestión, a través de la capacitación, asesoramiento, asistencia técnica y educación de los usuarios" : "Technical_assistance-Perú",
"Asesoría prestada al usuario por un operador acreditado, conducente a elaborar, acompañar y apoyar la adecuada ejecución técnica en terreno de aquellas prácticas comprometidas en el Plan de Manejo" : "Technical_assistance-Chile",
"Para la ejecución de programas de capacitación, adiestramiento y otorgamiento de becas para la preparación de personal , así como para el desarrollo de tecnología en actividades directamente relacionadas con las operaciones objeto del contrato" : "Technical_assistance-Guatemala",
"Apoyo técnico y en formulación de proyectos y conexión con mercados" : "Technical_assistance-El Salvador"}

queries = []
for query in queries_dict:
    queries.append(query)
        
# print(queries)

In [15]:
check_dictionary_values(queries_dict)

{'Credit': 0, 'Direct_payment': 0, 'Fine': 0, 'Guarantee': 0, 'Supplies': 0, 'Tax_deduction': 0, 'Technical_assistance': 0}
{'México': 0, 'Perú': 0, 'Chile': 0, 'Guatemala': 0, 'El Salvador': 0}


In [16]:
transformer_name ='xlm-r-bert-base-nli-stsb-mean-tokens'
similarity_threshold = 0.2
search_results_limit = 1000
today = datetime.date.today()
today = today.strftime('%Y-%m-%d')
# name = "Pre_tagged_" + today + "_" + filter_language
name = "Pre_tagged_" + today

model = SentenceTransformer(transformer_name)
results_dict = sentence_similarity_search(model, queries, embs, sample_sentences, similarity_threshold, search_results_limit, name)

similarity search for query Otorgamiento de estímulos crediticios por parte de el estado has been done in 0.1566 seconds
similarity search for query Estos créditos podrían beneficiar a sistemas productivos asociados a la pequeña y mediana producción has been done in 0.0137 seconds
similarity search for query Se asocia con créditos de enlace del Banco del Estado has been done in 0.0130 seconds
similarity search for query Acceso al programa de garantía crediticia para la actividad económica has been done in 0.0129 seconds
similarity search for query El banco establecerá líneas de crédito para que el sistema financiero apoye la pequeña, mediana y microempresa has been done in 0.0128 seconds
similarity search for query Dentro de los incentivos económicos se podrá crear un bono para retribuir a los propietarios por los bienes y servicios generados. has been done in 0.0135 seconds
similarity search for query Acceso a los fondos forestales para el pago de actividad has been done in 0.0179 sec

In [17]:
path = "./data_augmentation/output/new/"
fname = name + ".json"
file = path + fname
with open(file, 'w+') as fp:
    json.dump(results_dict, fp, indent=4)

In [18]:
# To show the contents of the results dict, particularly, the length of the first element and its contents
def show_results(results_dictionary):
    i = 0
    for key1 in results_dictionary:
        for key2 in results_dictionary[key1]:
            if i == 0:
                print(len(results_dictionary[key1][key2]))
                print(results_dictionary[key1][key2])
            i += 1

# Adding the rank to each result
def add_rank(results_dictionary):
#     for model in results_dictionary:
    for keyword in results_dictionary:#[model]:
        i = 1
        for result in results_dictionary[keyword]:#[model][keyword]:
            result.insert(1, i)
            i += 1
    return results_dictionary

# For experiments 2 and 3 this function is to save results in separate csv files
def save_results_as_separate_csv(results_dictionary, queries_dictionary, date):
    path = "C:/Users/Allie/Documents/GitHub/policy-data-analyzer/tasks/data_augmentation/output/new/"
#     for model, value in results_dictionary.items():
    for exp_title, result in results_dictionary.items():#value.items():
        filename = queries_dictionary[exp_title]
        file = path + filename + ".csv"
        with open(file, 'w', newline='', encoding='utf-8') as f:
            write = csv.writer(f)
            write.writerows(result)
#             print(filename)
    

In [21]:
path = "C:/Users/Allie/Documents/GitHub/policy-data-analyzer/tasks/data_augmentation/output/new/"
file = path + name +".json"
with open(file, "r") as f:
    results_ = json.load(f)

In [22]:
results = add_rank(results_)
len(results_)

26

In [ ]:
# Adding the rank to the results dictionary if we use the computed version not the uploaded
#results = copy.deepcopy(add_rank(results_dict))

In [23]:
# Save the results as separete csv files

save_results_as_separate_csv(results, queries_dict, today)